In [1]:
import geopandas as gpd
import pandas as pd
import numpy as np
from scipy.spatial import KDTree
from sklearn.preprocessing import QuantileTransformer, StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from libpysal.graph import read_parquet

In [2]:
clusters_dir = '/data/uscuni-ulce/processed_data/clusters/'
chars_dir = '/data/uscuni-ulce/processed_data/chars/'
regions_datadir = "/data/uscuni-ulce/"
region_hulls = gpd.read_parquet(
        regions_datadir + "regions/" + "cadastre_regions_hull.parquet"
)
graph_dir = '/data/uscuni-ulce/processed_data/neigh_graphs/'
morphotopes_dir = '/data/uscuni-ulce/processed_data/morphotopes/'

In [3]:
region_id = 69333

In [4]:
input_model_params = '_75_0_None_None_False'

In [5]:
output_model_params = '_post_processing_v1'

# 1. Change morphotope boundaries so that adjacent buildings are always in the majority morphotope.

In [125]:
def post_process_morphotope_labels(region_id, input_model_params, output_model_params):
    
    ## read data
    bq1 = read_parquet(graph_dir + f"building_graph_{region_id}.parquet")
    region_morphotope_labels = pd.read_parquet(f'{morphotopes_dir}tessellation_labels_morphotopes_{region_id}{input_model_params}.pq').morphotope_label
    region_morphotope_labels = region_morphotope_labels.loc[bq1.unique_ids]
    
    # assign mode of non-noise clusters to whole adjacent structure
    # if its only noise, assign the most common noise cluster
    def non_noise_mode(x):
        non_noise = x[~x.str.endswith('-1')]
        if non_noise.shape[0]:
            return pd.Series.mode(non_noise)[0]
        else:
            return pd.Series.mode(x)[0]
            
    component_morphotopes = region_morphotope_labels.groupby(bq1.component_labels).agg(non_noise_mode)
    
    aggregated_morphotope_labels = bq1.component_labels.map(component_morphotopes.to_dict())
    aggregated_morphotope_labels.name = 'morphotope_label'
    assert (aggregated_morphotope_labels.index == region_morphotope_labels.index).all()
    aggregated_morphotope_labels.to_frame().to_parquet(f'{morphotopes_dir}tessellation_labels_morphotopes_{region_id}{output_model_params}.pq')

In [126]:
%%time
from joblib import Parallel, delayed
n_jobs = -1
new = Parallel(n_jobs=n_jobs)(
    delayed(post_process_morphotope_labels)(region_id, input_model_params, output_model_params) for region_id, _ in region_hulls.iterrows()
)

CPU times: user 2.77 s, sys: 1.55 s, total: 4.32 s
Wall time: 15min 3s


# 2. Generate new morphotope data based on the new morphotope boundaries

In [10]:
def percentile(n):
    def percentile_(x):
        return np.percentile(x, n)
    percentile_.__name__ = 'percentile_%s' % n
    return percentile_

def post_process_morphotope_data(region_id, input_model_params, output_model_params):
    ## read data
    new_morphotope_labels = pd.read_parquet(f'{morphotopes_dir}tessellation_labels_morphotopes_{region_id}{output_model_params}.pq').morphotope_label
    X_train = pd.read_parquet(chars_dir + f'primary_chars_{region_id}.parquet')
    component_data = X_train.loc[new_morphotope_labels.index]
    
    
    # get morphotope stats
    component_data = component_data.groupby(new_morphotope_labels.values).agg([percentile(25), 'median', percentile(75), 'std', 'mean'])
    
    # save sizes for clustering
    component_data[('Size', 'Size')] = X_train.loc[new_morphotope_labels.index].groupby(new_morphotope_labels.values).size()
    
    # store morphotopes data
    component_data.to_parquet(morphotopes_dir + f'data_morphotopes_{region_id}{output_model_params}.pq')

In [11]:
%%time
post_process_morphotope_data(region_id, input_model_params, output_model_params)

CPU times: user 41.3 s, sys: 540 ms, total: 41.9 s
Wall time: 41 s


In [12]:
%%time
from joblib import Parallel, delayed
n_jobs = -1
new = Parallel(n_jobs=n_jobs)(
    delayed(post_process_morphotope_data)(region_id, input_model_params, output_model_params) for region_id, _ in region_hulls.iterrows()
)

/home/krasen/urban_taxonomy/.pixi/envs/default/lib/python3.12/site-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


CPU times: user 2.83 s, sys: 1.16 s, total: 3.99 s
Wall time: 18min 17s


### Plotting

In [110]:

from lonboard import SolidPolygonLayer, Map
from lonboard.basemap import CartoBasemap
from lonboard.colormap import apply_categorical_cmap
from palettable.colorbrewer.qualitative import Set3_12
from core.cluster_validation import get_color

In [111]:
buildings = gpd.read_parquet(f'/data/uscuni-ulce/processed_data/buildings/buildings_{region_id}.parquet')

In [112]:
buildings['morph'] = region_morphotope_labels
buildings['new_morph'] = aggregated_morphotope_labels
buildings['component_label'] = bq1.component_labels

In [113]:
layer = SolidPolygonLayer.from_geopandas(
    gdf=buildings[["geometry", "morph", 'new_morph', 'component_label']], opacity=0.15
)

/home/krasen/urban_taxonomy/.pixi/envs/default/lib/python3.12/site-packages/lonboard/_geoarrow/ops/reproject.py:97: UserWarning: Input being reprojected to EPSG:4326 CRS
  warnings.warn("Input being reprojected to EPSG:4326 CRS")


In [114]:
m = Map(layer, basemap_style=CartoBasemap.Positron)
m

Map(basemap_style=<CartoBasemap.Positron: 'https://basemaps.cartocdn.com/gl/positron-gl-style/style.json'>, la…

In [115]:
factors, idx = buildings['new_morph'].factorize()

In [116]:
layer.get_fill_color = get_color(factors)

# Experiments

### Add spatial restrictions to morphotope deliniation

In [11]:
from libpysal.graph import Graph

output_model_params = '_post_processing_v2'

In [12]:
def relabel_morphotopes_geography(group):

    # extract morph info
    gc = group.morph.iloc[0].split('_')[0]
    morph_number = group.morph.iloc[0].split('_')[1]

    # gropu the buildings with no 100m distances between them
    focal, neighbor = group.sindex.query(group.geometry, predicate='dwithin', distance=100)
    geom_graph = Graph.from_adjacency(pd.DataFrame({'focal': group.index[focal],
                                                    'neighbor': group.index[neighbor], 'weight': 1}))
    value_counts = geom_graph.component_labels.value_counts()
    
    # return new morphotope labels - change format from 123_0 to 123_00
    comp_labels = geom_graph.component_labels.map(lambda x: f'{gc}_{morph_number}{str(x)}'  if value_counts[x] > 75 else f'{gc}_-1')
    
    return comp_labels

In [13]:
def post_process_morphotope_labels(region_id, input_model_params, output_model_params):
    
    ## read data
    bq1 = read_parquet(graph_dir + f"building_graph_{region_id}.parquet")
    region_morphotope_labels = pd.read_parquet(f'{morphotopes_dir}tessellation_labels_morphotopes_{region_id}{input_model_params}.pq').morphotope_label
    region_morphotope_labels = region_morphotope_labels.loc[bq1.unique_ids]
    
    # assign mode of non-noise clusters to whole adjacent structure
    # if its only noise, assign the most common noise cluster
    def non_noise_mode(x):
        non_noise = x[~x.str.endswith('-1')]
        if non_noise.shape[0]:
            return pd.Series.mode(non_noise)[0]
        else:
            return pd.Series.mode(x)[0]
            
    component_morphotopes = region_morphotope_labels.groupby(bq1.component_labels).agg(non_noise_mode)
    
    aggregated_morphotope_labels = bq1.component_labels.map(component_morphotopes.to_dict())
    aggregated_morphotope_labels.name = 'morphotope_label'

    del bq1
    
    # read buildings geometries
    buildings = gpd.read_parquet(f'/data/uscuni-ulce/processed_data/chars/buildings_chars_{region_id}.parquet', columns=['geometry'])
    buildings['morph'] = aggregated_morphotope_labels

    # tighten morphotopes based on geographic distances
    res = []
    groups = buildings.groupby('morph')
    for key, group in groups:
    
        # if noise leave unchanged
        if key.split('_')[-1] == '-1':
            res.append(group.morph)
            continue
        else:
            res.append(relabel_morphotopes_geography(group))
    
    aggregated_morphotope_labels = pd.concat(res).sort_index()
    aggregated_morphotope_labels.name = 'morphotope_label'

    
    assert (aggregated_morphotope_labels.index == region_morphotope_labels.index).all()
    aggregated_morphotope_labels.to_frame().to_parquet(f'{morphotopes_dir}tessellation_labels_morphotopes_{region_id}{output_model_params}.pq')

In [14]:
# %%time
# from joblib import Parallel, delayed
# n_jobs = -1
# new = Parallel(n_jobs=n_jobs)(
#     delayed(post_process_morphotope_labels)(region_id, input_model_params, output_model_params) for region_id, _ in region_hulls.iterrows()
# )

Add stats from the morphotope core - the most similar 75 ETCs

In [21]:
from scipy.spatial.distance import pdist
from sklearn.preprocessing import StandardScaler
from scipy.cluster.hierarchy import linkage, fcluster

In [28]:
def calculate_core_stats(group):

    # if its noise retrun whatever
    if (group.empty) or (group.shape[0] < 75):
        core_stats = group.describe().loc[['25%', '50%', '75%', 'std', 'mean']]
        core_stats.index = ['percentile_25', 'median', 'percentile_75', 'std', 'mean']
        return core_stats.stack().swaplevel().sort_index()
    
    # find the first 75 nearest ETCs - the morphotope core
    scaled = StandardScaler().fit_transform(group.values)
    lm = linkage(pdist(np.nan_to_num(scaled)))
    cutoff = lm[np.where(lm[:, 3] >= 75)[0][0], 2]

    clusters = fcluster(lm, t=cutoff, criterion='distance')
    core_cluster = pd.Series(clusters).value_counts().index[0]
    core_etcs = group[clusters == core_cluster]

    # calculate stats
    core_stats = core_etcs.describe().loc[['25%', '50%', '75%', 'std', 'mean']]
    core_stats.index = ['percentile_25', 'median', 'percentile_75', 'std', 'mean']
    return core_stats.stack().swaplevel().sort_index(level=0)
    

def post_process_morphotope_data(region_id, output_model_params):
    ## read data
    new_morphotope_labels = pd.read_parquet(f'{morphotopes_dir}tessellation_labels_morphotopes_{region_id}{output_model_params}.pq').morphotope_label
    X_train = pd.read_parquet(chars_dir + f'primary_chars_{region_id}.parquet')
    component_data = X_train.loc[new_morphotope_labels.index]
    
    
    # get morphotope stats
    component_data = component_data.groupby(new_morphotope_labels.values).apply(calculate_core_stats)
    component_data = component_data.unstack([1, 2])
    
    # save sizes for clustering
    component_data[('Size', 'Size')] = X_train.loc[new_morphotope_labels.index].groupby(new_morphotope_labels.values).size()
    
    # store morphotopes data
    component_data.to_parquet(morphotopes_dir + f'data_morphotopes_{region_id}_{output_model_params}.pq')

In [29]:
%%time
from joblib import Parallel, delayed
n_jobs = -1
new = Parallel(n_jobs=n_jobs)(
    delayed(post_process_morphotope_data)(region_id, output_model_params) for region_id, _ in region_hulls.iterrows()
)

/home/krasen/urban_taxonomy/.pixi/envs/default/lib/python3.12/site-packages/sklearn/utils/extmath.py:1101: RuntimeWarning: invalid value encountered in divide
  updated_mean = (last_sum + new_sum) / updated_sample_count
/home/krasen/urban_taxonomy/.pixi/envs/default/lib/python3.12/site-packages/sklearn/utils/extmath.py:1106: RuntimeWarning: invalid value encountered in divide
  T = new_sum / new_sample_count
/home/krasen/urban_taxonomy/.pixi/envs/default/lib/python3.12/site-packages/sklearn/utils/extmath.py:1126: RuntimeWarning: invalid value encountered in divide
  new_unnormalized_variance -= correction**2 / new_sample_count
/home/krasen/urban_taxonomy/.pixi/envs/default/lib/python3.12/site-packages/sklearn/utils/extmath.py:1101: RuntimeWarning: invalid value encountered in divide
  updated_mean = (last_sum + new_sum) / updated_sample_count
/home/krasen/urban_taxonomy/.pixi/envs/default/lib/python3.12/site-packages/sklearn/utils/extmath.py:1106: RuntimeWarning: invalid value encounte

MemoryError: Unable to allocate 267. GiB for an array with shape (35829101895,) and data type float64

In [27]:
# region_id = 69333
# post_process_morphotope_data(region_id, output_model_params)

In [18]:
## read data
new_morphotope_labels = pd.read_parquet(f'{morphotopes_dir}tessellation_labels_morphotopes_{region_id}{output_model_params}.pq').morphotope_label
X_train = pd.read_parquet(chars_dir + f'primary_chars_{region_id}.parquet')
component_data = X_train.loc[new_morphotope_labels.index]

In [19]:
groups = component_data.groupby(new_morphotope_labels)


In [106]:
pd.read_parquet(morphotopes_dir + f'data_morphotopes_{region_id}_post_processing_v1.pq')

sdbAre                                                    \
        percentile_25      median percentile_75         std        mean   
0_-1       137.301434  137.301434    137.301434         NaN  137.301434   
1000_0     109.759359  145.329083    216.693998  347.022351  218.154302   
1001_-1     43.816714  129.538040    163.491878   70.727784  111.560013   
1002_-1    119.777291  122.679492    125.581693    8.208663  122.679492   
1003_-1    219.429978  249.432694    279.435409   84.860494  249.432694   
...               ...         ...           ...         ...         ...   
998_-1      31.203731   31.203731     31.203731         NaN   31.203731   
999_-1      33.352216   45.950070     55.400582   41.333709   53.902909   
99_-1       55.810224   55.810224     55.810224         NaN   55.810224   
9_-1       113.026756  151.083435    234.450644  427.444239  276.386584   
9_0         97.262678  136.332740    194.229942   81.952598  153.495672   

               sdbPer                                                 ...  \
        percentile_25     median percentile_75        std       mean  ...   
0_-1        49.637208  49.637208     49.637208        NaN  49.637208  ...   
1000_0      45.067388  53.718509     73.310913  35.140641  64.173489  ...   
1001_-1     26.995417  52.926018     59.971880  20.294192  46.359191  ...   
1002_-1     48.152608  48.227610     48.302612   0.212138  48.227610  ...   
1003_-1     65.926905  71.563066     77.199228  15.941473  71.563066  ...   
...               ...        ...           ...        ...        ...  ...   
998_-1      22.401346  22.401346     22.401346        NaN  22.401346  ...   
999_-1      23.497498  27.661601     31.683057  11.509662  30.022516  ...   
99_-1       29.883778  29.883778     29.883778        NaN  29.883778  ...   
9_-1        43.641113  58.379816     79.560283  76.395201  75.766057  ...   
9_0         42.894249  55.095135     69.560058  23.319055  58.763380  ...   

             misBAD                                              midBAD  \
             median percentile_75         std        mean percentile_25   
0_-1      52.456478     52.456478         NaN   52.456478    155.664723   
1000_0   151.417331           NaN  218.635071  183.540407    229.475859   
1001_-1   65.233299     67.534362    3.533038   64.430592     64.151064   
1002_-1   82.479491     82.479491    0.000000   82.479491     28.718283   
1003_-1   82.479491     82.479491    0.000000   82.479491     88.026361   
...             ...           ...         ...         ...           ...   
998_-1    59.721408     59.721408         NaN   59.721408     59.721408   
999_-1    59.721408     59.721408    0.000000   59.721408     59.721408   
99_-1     73.615914     73.615914         NaN   73.615914     77.460818   
9_-1      72.955090           NaN  356.747605  230.454381     45.366236   
9_0       78.065821           NaN  127.184195  101.465381     69.745933   

                                                          Size  
             median percentile_75         std        mean Size  
0_-1     155.664723    155.664723         NaN  155.664723    1  
1000_0   229.475859    579.241962  197.279745  285.671852   86  
1001_-1   69.365213     73.102810    6.004547   67.888661    8  
1002_-1   28.718283     28.718283    0.000000   28.718283    2  
1003_-1   88.026361     88.026361    0.000000   88.026361    2  
...             ...           ...         ...         ...  ...  
998_-1    59.721408     59.721408         NaN   59.721408    1  
999_-1    59.721408     59.721408    0.000000   59.721408   58  
99_-1     77.460818     77.460818         NaN   77.460818    1  
9_-1      88.360506    592.692346  321.118785  254.324753   54  
9_0       76.905519    123.550145  196.194703  146.301650  104  

[6134 rows x 381 columns]

In [107]:
group = groups.get_group('849_3610')

In [157]:
def calculate_core_stats(group):

    # if its noise retrun whatever
    if (group.empty) or (group.shape[0] < 75):
        core_stats = group.describe().loc[['25%', '50%', '75%', 'std', 'mean']]
        core_stats.index = ['percentile_25', 'median', 'percentile_75', 'std', 'mean']
        return core_stats.stack().swaplevel().sort_index()
    
    # find the first 75 nearest ETCs - the morphotope core
    scaled = StandardScaler().fit_transform(group.values)
    lm = linkage(pdist(np.nan_to_num(scaled)))
    cutoff = lm[np.where(lm[:, 3] >= 75)[0][0], 2]

    clusters = fcluster(lm, t=cutoff, criterion='distance')
    core_cluster = pd.Series(clusters).value_counts().index[0]
    core_etcs = group[clusters == core_cluster]

    # calculate stats
    core_stats = core_etcs.describe().loc[['25%', '50%', '75%', 'std', 'mean']]
    core_stats.index = ['percentile_25', 'median', 'percentile_75', 'std', 'mean']
    return core_stats.stack().swaplevel()

In [158]:
res = calculate_core_stats(group)

In [160]:
%%time
res = groups.apply(calculate_core_stats)

CPU times: user 3min 24s, sys: 176 ms, total: 3min 24s
Wall time: 3min 24s


In [161]:
res = res.unstack([1, 2])

In [162]:
res[res.index.str.split('_').str[-1] != '-1']

lcdMes                                        \
                      mean    median percentile_25 percentile_75   
morphotope_label                                                   
1000_00           0.045560  0.041096      0.041096      0.047619   
1005_00           0.082474  0.082474      0.082474      0.082474   
1016_00           0.114720  0.112360      0.102804      0.145299   
1016_10           0.113773  0.111111      0.107692      0.123457   
1016_20           0.145962  0.144578      0.126984      0.161905   
...                    ...       ...           ...           ...   
978_00            0.091688  0.089552      0.084507      0.101266   
991_00            0.080440  0.080000      0.068966      0.093333   
991_10            0.104042  0.106383      0.085714      0.122449   
997_00            0.122255  0.140351      0.096169      0.140351   
9_00              0.151821  0.156863      0.147541      0.170732   

                        lcnClo                                            \
                          mean        median percentile_25 percentile_75   
morphotope_label                                                           
1000_00           1.692468e-07  1.844877e-07  1.556789e-07  1.844877e-07   
1005_00           2.923115e-07  2.923115e-07  2.923115e-07  2.923115e-07   
1016_00           1.004474e-06  1.067629e-06  8.929103e-07  1.301400e-06   
1016_10           1.069447e-06  1.000224e-06  8.622150e-07  1.183166e-06   
1016_20           1.709577e-06  1.689486e-06  1.578546e-06  2.003597e-06   
...                        ...           ...           ...           ...   
978_00            1.310054e-06  1.204307e-06  1.094749e-06  1.662271e-06   
991_00            5.112157e-07  5.199192e-07  3.465594e-07  6.848443e-07   
991_10            5.633065e-07  4.846408e-07  4.238164e-07  7.234224e-07   
997_00            2.975942e-07  2.949158e-07  2.723503e-07  3.156148e-07   
9_00              3.158303e-07  3.061929e-07  2.749514e-07  3.675227e-07   

                     ldbPWL             ...    ssbSqu    sscCCo    sscERI  \
                       mean     median  ...       std       std       std   
morphotope_label                        ...                                 
1000_00           68.769858  63.653309  ...  2.766928  0.124801  0.083728   
1005_00           58.625253  53.237479  ...  2.549229  0.115867  0.066186   
1016_00           54.210444  51.505762  ...  4.567328  0.113109  0.054705   
1016_10           47.323900  47.432468  ...  3.114346  0.102199  0.057288   
1016_20           42.585157  44.866511  ...  2.877012  0.107088  0.049552   
...                     ...        ...  ...       ...       ...       ...   
978_00            54.115970  51.694544  ...  2.046621  0.113848  0.059074   
991_00            52.395414  49.923359  ...  3.902263  0.118226  0.054350   
991_10            46.918799  45.098962  ...  1.172118  0.122181  0.073687   
997_00            51.501311  49.064966  ...  4.281989  0.110449  0.059296   
9_00              63.912768  61.062566  ...  2.103256  0.095712  0.058199   

                    sssLin     stbCeA     stbOri     stbSAl     stcOri  \
                       std        std        std        std        std   
morphotope_label                                                         
1000_00           0.026213   8.910203  11.906003   9.091221  12.335940   
1005_00           0.034214   8.777150  10.696158   6.051219  12.113448   
1016_00           0.027963   8.039746  11.421928   7.365768  12.526173   
1016_10           0.329734   7.536924  10.981749  10.362463  12.015506   
1016_20           0.070840   6.346112   7.123301   5.497911   9.823816   
...                    ...        ...        ...        ...        ...   
978_00            0.029332   9.027582   9.776263   8.188248  12.530653   
991_00            0.036807  10.096105   9.327904   7.507936  13.971638   
991_10            0.059951   9.076529  13.079530   7.727762  13.544423   
997_00            0.036932  10.2671

In [51]:
buildings = gpd.read_parquet(f'/data/uscuni-ulce/processed_data/chars/buildings_chars_{region_id}.parquet', columns=['geometry'])


In [54]:
m = buildings.loc[group.index].explore()
m = buildings.loc[core_etcs.index].explore(m=m, color='r')
m

In [64]:
## read data
bq1 = read_parquet(graph_dir + f"building_graph_{region_id}.parquet")
region_morphotope_labels = pd.read_parquet(f'{morphotopes_dir}tessellation_labels_morphotopes_{region_id}{input_model_params}.pq').morphotope_label
region_morphotope_labels = region_morphotope_labels.loc[bq1.unique_ids]

# assign mode of non-noise clusters to whole adjacent structure
# if its only noise, assign the most common noise cluster
def non_noise_mode(x):
    non_noise = x[~x.str.endswith('-1')]
    if non_noise.shape[0]:
        return pd.Series.mode(non_noise)[0]
    else:
        return pd.Series.mode(x)[0]
        
component_morphotopes = region_morphotope_labels.groupby(bq1.component_labels).agg(non_noise_mode)

aggregated_morphotope_labels = bq1.component_labels.map(component_morphotopes.to_dict())
aggregated_morphotope_labels.name = 'morphotope_label'

In [99]:
%%time

buildings = gpd.read_parquet(f'/data/uscuni-ulce/processed_data/chars/buildings_chars_{region_id}.parquet', columns=['geometry'])
buildings['morph'] = aggregated_morphotope_labels

res = []
groups = buildings.groupby('morph')
for key, group in groups:

    # if noise leave unchanged
    if key.split('_')[-1] == '-1':
        res.append(group.morph)
        continue
    else:
        res.append(relabel_morphotopes_geography(group))

aggregated_morphotope_labels = pd.concat(res).sort_index()
aggregated_morphotope_labels.name = 'morphotope_label'

CPU times: user 22.4 s, sys: 15.6 ms, total: 22.4 s
Wall time: 22.4 s


In [103]:
aggregated_morphotope_labels

focal
0            0_-1
1             1_0
2             1_0
3             1_0
4             1_0
           ...   
446822    849_640
446823     849_-1
446824    849_487
446825    849_508
446826    849_361
Name: morphotope_label, Length: 446827, dtype: object

In [95]:
def relabel_morphotopes_geography(group):

    # extract morph info
    gc = group.morph.iloc[0].split('_')[0]
    morph_number = group.morph.iloc[0].split('_')[1]

    # gropu the buildings with no 100m distances between them
    focal, neighbor = group.sindex.query(group.geometry, predicate='dwithin', distance=100)
    geom_graph = Graph.from_adjacency(pd.DataFrame({'focal': group.index[focal],
                                                    'neighbor': group.index[neighbor], 'weight': 1}))
    value_counts = geom_graph.component_labels.value_counts()
    
    # return new morphotope labels
    comp_labels = geom_graph.component_labels.map(lambda x: f'{gc}_{morph_number}_{str(x)}'  if value_counts[x] > 75 else f'{gc}_-1')
    
    return comp_labels

In [30]:
# %%time
# res = buildings.groupby('morph').transform(relabel_morphotopes_geography)